In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# recfldgrn List


In this section, we want to get all available `rec@fld_grn list`.

In [2]:
# load some data reader function from recfldgrn module.
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
import os

Load the raw tensor (idx) from a TaskTensor Folder.

Here we treat the Task as given (the cohort, X, and Y are ready). 

Here the task name is `Task2YearXXX` (this name is meaningless, just for demonstration).

At the end, we have `recfldgrn_list` which shows all available `rec@fld_grn` we have.

In [3]:
Tensor_folder = 'data/ProcData/FldGrnTensor/'
print(Tensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(Tensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/FldGrnTensor/


['P@PatEcInfo-InfoGrn', 'PNSect@AllText-TknzGrn']

# One recfldgrn

Here we focus on one `rec@fld_grn` to check how we parse the information.

In the python variable name, we use `recfldgrn` to refer to `rec@fld_grn`.


## Load df_Pat

In [4]:
##############
# recfldgrn = 'PNSectSent@Sentence-Tk@TknzLLMGrn'
# recfldgrn = 'A1C@DT-DTDftGrn'
recfldgrn = 'PNSect@AllText-TknzGrn'
##############

In [5]:
# tensor_folder, get the folder of the focal recfldgrn
tensor_folder = os.path.join(Tensor_folder, recfldgrn)
print(tensor_folder, '<---- tensor_folder')

# get the df_Pat: the recfldgrn grouped in the Pat (patient) level.
df_tensor_fnl = load_df_data_from_folder(tensor_folder)# .set_index('PID')
print(df_tensor_fnl.shape, '<---- the shape of df_tensor_fnl at top level')

df_tensor_fnl


data/ProcData/FldGrnTensor/PNSect@AllText-TknzGrn <---- tensor_folder
(8, 4) <---- the shape of df_tensor_fnl at top level


,PID,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
0,P0,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
1,P1,"[[[[101, 3558, 11360, 102, 101, 3056, 3034, 43...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,P2,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
3,P3,"[[[[101, 3558, 11360, 102, 101, 3193, 4769, 19...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,P4,"[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,P5,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
6,P6,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
7,P7,"[[[[101, 2708, 12087, 102, 101, 2482, 2995, 26...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2..."


# Build Dataset

In [6]:
# How to Get this?
Elig_Set = ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [7]:
df_tensor_fnl = df_tensor_fnl[df_tensor_fnl['PID'].isin(Elig_Set)].reset_index(drop = True)
df_tensor_fnl

,PID,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
0,P0,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
1,P1,"[[[[101, 3558, 11360, 102, 101, 3056, 3034, 43...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,P2,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
3,P3,"[[[[101, 3558, 11360, 102, 101, 3193, 4769, 19...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,P4,"[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,P5,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
6,P6,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
7,P7,"[[[[101, 2708, 12087, 102, 101, 2482, 2995, 26...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2..."


## get Batch

In [8]:
###################### take this as given
batch_PID_order = ['P0', 'P2', # 'P1', 'P3', 
                   'P4', 'P5', # 'P6', 'P7'
                  ]
######################

In [9]:
# generate a batch
df_batch = df_tensor_fnl.set_index('PID').loc[batch_PID_order]
df_batch

,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
PID,,,
P0,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P2,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P4,"[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P5,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."


In [10]:
# df_batch[full_recfldgrn].to_list()

In [11]:
recfldgrn_sfx_list = list(df_batch.columns)
recfldgrn_sfx_list

['P-EC-PN-PNSect@AllText-TknzGrn_tknidx',
 'P-EC-PN-PNSect@AllText-TknzGrn_wgt',
 'P-EC-PN-PNSect@AllText-TknzGrn_fldidx']

In [12]:
# get the relational_list
for recfldgrn_sfx in recfldgrn_sfx_list:
    print(recfldgrn_sfx, '<--- full_recfldgrn')
    relational_list = df_batch[recfldgrn_sfx].to_list()
    
    # add a batch prefix to the old full_recfldgrn
    B_recfldgrn_sfx = 'B-' + recfldgrn_sfx # B- means Batch. 
    print(B_recfldgrn_sfx, '<--- B_recfldgrn_sfx')
    suffix = '_' + B_recfldgrn_sfx.split('_')[-1]
    print(suffix, '<---- suffix\n')

P-EC-PN-PNSect@AllText-TknzGrn_tknidx <--- full_recfldgrn
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx <--- B_recfldgrn_sfx
_tknidx <---- suffix

P-EC-PN-PNSect@AllText-TknzGrn_wgt <--- full_recfldgrn
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt <--- B_recfldgrn_sfx
_wgt <---- suffix

P-EC-PN-PNSect@AllText-TknzGrn_fldidx <--- full_recfldgrn
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx <--- B_recfldgrn_sfx
_fldidx <---- suffix



In [13]:
# have a look at df_batch
df_batch

,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
PID,,,
P0,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P2,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P4,"[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P5,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."


## convert_relational_list_to_numpy

In [14]:
# these two functions want to convert the tensor from a nested relational_list to array.
# so we need to padding zeros to all the lists in order to make an array.
# the function `traverse` and `convert_relational_list_to_numpy` will do this for you. 

from fieldnn.utils.layerfn import traverse
from fieldnn.utils.layerfn import convert_relational_list_to_numpy


def convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix):
    o = values_list
    layer_num = len(new_full_recfldgrn.split('-'))
    layers = new_full_recfldgrn.replace(suffix, '').split('-')
    # L = [len(values_list)] 

    D = {}

    # (1) from first layer: 0
    idx = 0
    layer_parents = layers[:idx + 1]
    layer_children = layers[idx + 1]
    len_name = f'{"-".join(layer_parents)}_ln{layer_children}'
    len_np = np.array(len(values_list))
    len_shapes = [len_np.max()] # from layer 0, prepare for layer 1. 
    D[len_name] = len_np

    # (2) from 1 - last one layers
    for idx in range(1, layer_num - 1):
        output = list(traverse(o, nest_layer = idx))
        # print(output)
        # data = np.zeros(L)
        # print('\n\n')
        # print(idx)
        # print(output)

        layer_parents = layers[:idx + 1]
        layer_children = layers[idx + 1]
        len_name = f'{"-".join(layer_parents)}_ln{layer_children}'
        # print(len_name)

        locidx  = [i[0] for i in output]
        length = [i[1] for i in output]
        # values = [i[2] for i in output]

        len_np = np.zeros(len_shapes).astype(int)
        # print(len_np.shape, '<---- len_np.shape')
        for locidx, length, _ in output:
            len_np[tuple(locidx)] = int(length)
        # print(len_np)
        len_shapes.append(len_np.max())
        # print(len_shapes, '<---- next len_np.shape')
        # print(length)
        # print()
        D[len_name] = len_np

    # (3) for the data
    idx = layer_num
    name = new_full_recfldgrn
    data = np.zeros(len_shapes) # don't convert it to int for now. 
    output = list(traverse(o, nest_layer = idx))
    for locidx, _, value in output:
        data[tuple(locidx)] = value
        # print(locidx, value)
        # print(data[tuple(locidx)])# = value
    data.shape
    D[name] = data.astype(int)
    
    return D

In [15]:
import numpy as np
# padding the relatinal list to numpy


# inputs: you can check the following inputs in the above cells.
# (1): relational_list
# (2): B_recfldgrn_suffix
# (3): suffix

batch_rfg = {}

# get the relational_list
for recfldgrn_sfx in recfldgrn_sfx_list:
    # print(recfldgrn_suffix, '<--- recfldgrn_suffix')
    relational_list = df_batch[recfldgrn_sfx].to_list()
    # add a batch prefix to the old full_recfldgrn
    B_recfldgrn_sfx = 'B-' + recfldgrn_sfx # B- means Batch. 
    # print(B_recfldgrn_suffix, '<--- new_full_recfldgrn')
    suffix = '_' + B_recfldgrn_sfx.split('_')[-1]
    # print(suffix, '<---- suffix\n')

    D = convert_relational_list_to_numpy(relational_list, 
                                         B_recfldgrn_sfx, 
                                         suffix)

    # Here: k is the array name, v is the array
    # B_lnP: ln means the leng. B_lnP is a scalar (zero-order). This means how many patients in the batch
    # B-P_lnEC: a one-order array, each element is patient's EC number.
    # every array name with ln means the length. 
    # The last one is important: B-P-EC-A1C@DT-DTDftGrn_idx: the real value.
    # actually, in the real practice, we will only use the last one: i.e. B-P-EC-A1C@DT-DTDftGrn_idx. 
    # for k, v in D.items(): print(k, v.shape)
    # D2 = {}
    # for k, v in D.items(): 
    #     D2[k.replace('_', '@')] = v


    for k, v in D.items(): print(k, v.shape)
    
    # len of shape
    
    
    # have a look at B-P-EC-A1C@DT-DTDftGrn_idx, the final tensor
    tensor_idx = D[B_recfldgrn_sfx]
    # print(B_recfldgrn_sfx, '<--- B_recfldgrn_suffix')
    # print(tensor_idx.shape, '<------- the shape of tensor_idx')
    print('\n')
    
    
    batch_rfg[B_recfldgrn_sfx] = tensor_idx


B_lnP ()
B-P_lnEC (4,)
B-P-EC_lnPN (4, 34)
B-P-EC-PN_lnPNSect@AllText (4, 34, 1)
B-P-EC-PN-PNSect@AllText_lnTknzGrn (4, 34, 1, 14)
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx (4, 34, 1, 14, 283)


B_lnP ()
B-P_lnEC (4,)
B-P-EC_lnPN (4, 34)
B-P-EC-PN_lnPNSect@AllText (4, 34, 1)
B-P-EC-PN-PNSect@AllText_lnTknzGrn (4, 34, 1, 14)
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt (4, 34, 1, 14, 283)


B_lnP ()
B-P_lnEC (4,)
B-P-EC_lnPN (4, 34)
B-P-EC-PN_lnPNSect@AllText (4, 34, 1)
B-P-EC-PN-PNSect@AllText_lnTknzGrn (4, 34, 1, 14)
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx (4, 34, 1, 14, 283)




## Validation

Here we want to do a validation to check whether the returned padding array is the same value as the origianl relational list. 

In [16]:
# B_recfldgrn_sfx_list

In [17]:
recfldgrn_sfx = recfldgrn_sfx_list[0]
B_recfldgrn_sfx = 'B-' + recfldgrn_sfx
print(B_recfldgrn_sfx)
tensor_idx = batch_rfg[B_recfldgrn_sfx]

B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx


In [18]:
tensor_idx[1][9][0][0]

array([ 101, 2740, 6032,  102,  101, 3926, 2668, 4708, 3959, 3604, 1012,
       5293, 2236, 2606, 2215, 2604, 3005, 3749, 1012, 2742, 4067, 2009,
       2767, 2943, 4368, 1012, 2562, 3746, 2465, 1012, 2090, 3119, 4950,
       1012, 4072, 3189, 2694, 3899, 6570, 5060, 2933, 1012, 3392, 2507,
       3413, 2166, 1012, 4009, 3265, 3350, 3522, 4366, 2304, 1012,  102,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [19]:
relational_list = df_batch[recfldgrn_sfx].to_list()
print(relational_list[1][9][0][0])

[101, 2740, 6032, 102, 101, 3926, 2668, 4708, 3959, 3604, 1012, 5293, 2236, 2606, 2215, 2604, 3005, 3749, 1012, 2742, 4067, 2009, 2767, 2943, 4368, 1012, 2562, 3746, 2465, 1012, 2090, 3119, 4950, 1012, 4072, 3189, 2694, 3899, 6570, 5060, 2933, 1012, 3392, 2507, 3413, 2166, 1012, 4009, 3265, 3350, 3522, 4366, 2304, 1012, 102]


In [20]:
tensor_idx[3][7][0][0]

array([  101,  2053,  2930, 11746,   102,   101,  2053,  2930, 11746,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [21]:
relational_list = df_batch[recfldgrn_sfx].to_list()
relational_list[3][7][0][0]

[101, 2053, 2930, 11746, 102, 101, 2053, 2930, 11746, 102]

# All recfldgrn

In [22]:
###################### take this as given
batch_PID_order = ['P0', 'P2', # 'P1', 'P3', 
                   'P4', 'P5', # 'P6', 'P7'
                  ]
######################

In [23]:
Tensor_folder = 'data/ProcData/FldGrnTensor/'
print(Tensor_folder)
recfldgrn_list = sorted([i for i in os.listdir(Tensor_folder) if 'Grn' in i])
recfldgrn_list

data/ProcData/FldGrnTensor/


['P@PatEcInfo-InfoGrn', 'PNSect@AllText-TknzGrn']

## Choose a RecFldGrn List

In [24]:
# take this as given.
# we want to test whether these recfldgrn are important. 

recfldgrn_list = ['P@PatEcInfo-InfoGrn', 'PNSect@AllText-TknzGrn']

## Get df_batch

In [25]:
# How to Get this?
from functools import reduce
import pandas as pd


Elig_Set = ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']


L = []
for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(Tensor_folder, recfldgrn)
    # (2) get df_Pat and full_recfldgrn
    df_tensor_fnl = load_df_data_from_folder(tensor_folder)# .set_index('PID')
    # (2+) get your Eligible User
    df_tensor_fnl = df_tensor_fnl[df_tensor_fnl['PID'].isin(Elig_Set)].reset_index(drop = True).set_index('PID') 
    
    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_tensor_fnl.loc[batch_PID_order]
    L.append(df_batch)
    
df_batch = reduce(lambda left, right: pd.merge(left, right, on='PID'), L)
df_batch

,P@PatEcInfo-InfoGrn_wgt,P@PatEcInfo-InfoGrn_tknidx,P@PatEcInfo-InfoGrn_fldidx,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
PID,,,,,,
P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."


In [26]:
# a = L[0]
# b = L[1]
# pd.merge(a, b, on = 'PID')

In [27]:
recfldgrn_sfx_list = list(df_batch.columns)
recfldgrn_sfx_list

['P@PatEcInfo-InfoGrn_wgt',
 'P@PatEcInfo-InfoGrn_tknidx',
 'P@PatEcInfo-InfoGrn_fldidx',
 'P-EC-PN-PNSect@AllText-TknzGrn_tknidx',
 'P-EC-PN-PNSect@AllText-TknzGrn_wgt',
 'P-EC-PN-PNSect@AllText-TknzGrn_fldidx']

In [28]:
batch_rfg = {}

# get the relational_list
for recfldgrn_sfx in recfldgrn_sfx_list:
    relational_list = df_batch[recfldgrn_sfx].to_list()
    B_recfldgrn_sfx = 'B-' + recfldgrn_sfx # B- means Batch. )
    suffix = '_' + B_recfldgrn_sfx.split('_')[-1]
    D = convert_relational_list_to_numpy(relational_list, B_recfldgrn_sfx, suffix)
    tensor_idx = D[B_recfldgrn_sfx]
    batch_rfg[B_recfldgrn_sfx] = tensor_idx


In [29]:
for k, v in batch_rfg.items(): 
    print(k, v.shape)

B-P@PatEcInfo-InfoGrn_wgt (4, 43)
B-P@PatEcInfo-InfoGrn_tknidx (4, 43)
B-P@PatEcInfo-InfoGrn_fldidx (4, 43)
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx (4, 34, 1, 14, 283)
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt (4, 34, 1, 14, 283)
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx (4, 34, 1, 14, 283)


# Develop Dataset and DataLoader

We need to define a PyTorch Dataset and DataLoader to generate the batch automatically, based on the information we already have. This function/class will be developed based on the functions described in the following part. 


Input:
- 1. A task tensor folder, where you can load df_Pat for each `recfldgrn`.
- 2. A given list of recfldgrn.


TODO:

- 1. Design a PyTorch Dataset, where X: batch_rfg (only one patient).  Y: 'TODO' (return a the string of 'TODO' for now'. DONE!


- 2. Design a PyTorch DataLoader, Define `collate_fn`: https://pytorch.org/docs/stable/data.html#working-with-collate-fn that use the function of `convert_relational_list_to_numpy` to padding the  relational list into array.






In [30]:
recfldgrn_list

['P@PatEcInfo-InfoGrn', 'PNSect@AllText-TknzGrn']

In [31]:
print(len(Elig_Set))

8


In [32]:
Tensor_folder = 'data/ProcData/FldGrnTensor/'

for recfldgrn in recfldgrn_list:    
    # (1) get tensor_folder
    tensor_folder = os.path.join(Tensor_folder, recfldgrn)
    # (2) get df_Pat and full_recfldgrn
    df_tensor_fnl = load_df_data_from_folder(tensor_folder)# .set_index('PID')
    # (2+) get your Eligible User
    df_tensor_fnl = df_tensor_fnl[df_tensor_fnl['PID'].isin(Elig_Set)].reset_index(drop = True).set_index('PID') 

    tensor_cols = df_tensor_fnl.columns

In [33]:
df_tensor_fnl

,P-EC-PN-PNSect@AllText-TknzGrn_tknidx,P-EC-PN-PNSect@AllText-TknzGrn_wgt,P-EC-PN-PNSect@AllText-TknzGrn_fldidx
PID,,,
P0,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P1,"[[[[101, 3558, 11360, 102, 101, 3056, 3034, 43...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P2,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P3,"[[[[101, 3558, 11360, 102, 101, 3193, 4769, 19...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P4,"[[[[101, 2740, 6032, 102, 101, 2036, 3115, 316...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
P5,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P6,"[[[[101, 2053, 2930, 11746, 102, 101, 2053, 29...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], [[[1, 1, ...","[[[[2, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], [[[2, 2, ..."
P7,"[[[[101, 2708, 12087, 102, 101, 2482, 2995, 26...","[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1...","[[[[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2..."


## RFG Dataset

In [34]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import pandas as pd
from functools import reduce


class RFGDataset(Dataset):
    def __init__(self, Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID'):
        self.recfldgrn_list = recfldgrn_list
        self.Tensor_folder = Tensor_folder
        self.Elig_Set = Elig_Set
        self.RecRootID = RecRootID
        
        L = []
        for recfldgrn in recfldgrn_list:
            # (1) get tensor_folder
            tensor_folder = os.path.join(Tensor_folder, recfldgrn)
            # (2) get df_Pat and full_recfldgrn
            df_tensor_fnl = load_df_data_from_folder(tensor_folder)
            df_tensor_fnl = df_tensor_fnl[df_tensor_fnl[RecRootID].isin(Elig_Set)].reset_index(drop = True).set_index(RecRootID) 
            L.append(df_tensor_fnl)
        data = reduce(lambda left, right: pd.merge(left, right, on=RecRootID), L)
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index]# [full_recfldgrn]
        y = np.random.choice([0,1]) # go back to label later.
        return {'x': x, 'y': y} # torch.tensor(x), torch.tensor(y)
    
dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
# len(dataset)

In [35]:
len(dataset)

8

## Collate_FN

In [36]:
batch_input = [dataset[i] for i in [1, 3, 4, 6]]
# batch_input

In [37]:
df_batch = pd.DataFrame([i['x'].to_dict() for i in batch_input])
recfldgrn_sfx_list = [i for i in df_batch.columns]
recfldgrn_sfx_list

['P@PatEcInfo-InfoGrn_wgt',
 'P@PatEcInfo-InfoGrn_tknidx',
 'P@PatEcInfo-InfoGrn_fldidx',
 'P-EC-PN-PNSect@AllText-TknzGrn_tknidx',
 'P-EC-PN-PNSect@AllText-TknzGrn_wgt',
 'P-EC-PN-PNSect@AllText-TknzGrn_fldidx']

In [38]:
# here batch_rfg is dict to hold the padding array for each recfldgrn
batch_rfg = {}

for recfldgrn_sfx in recfldgrn_sfx_list:
    relational_list = df_batch[recfldgrn_sfx].to_list()
    B_recfldgrn_sfx = 'B-' + recfldgrn_sfx # B- means Batch. 
    suffix = '_' + B_recfldgrn_sfx.split('_')[-1]
    D = convert_relational_list_to_numpy(relational_list, B_recfldgrn_sfx, suffix)
    tensor_idx = D[B_recfldgrn_sfx]
    print(B_recfldgrn_sfx, '<--- B_recfldgrn_suffix')
    print(tensor_idx.shape, '<------- the shape of tensor_idx')
    batch_rfg[B_recfldgrn_sfx] = tensor_idx
    print('\n')

B-P@PatEcInfo-InfoGrn_wgt <--- B_recfldgrn_suffix
(4, 43) <------- the shape of tensor_idx


B-P@PatEcInfo-InfoGrn_tknidx <--- B_recfldgrn_suffix
(4, 43) <------- the shape of tensor_idx


B-P@PatEcInfo-InfoGrn_fldidx <--- B_recfldgrn_suffix
(4, 43) <------- the shape of tensor_idx


B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx <--- B_recfldgrn_suffix
(4, 87, 1, 15, 517) <------- the shape of tensor_idx


B-P-EC-PN-PNSect@AllText-TknzGrn_wgt <--- B_recfldgrn_suffix
(4, 87, 1, 15, 517) <------- the shape of tensor_idx


B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx <--- B_recfldgrn_suffix
(4, 87, 1, 15, 517) <------- the shape of tensor_idx




In [39]:
def my_collate_fn(batch_input):
    ##############
    # inputs: you can check the following inputs in the above cells.
    # (1): relational_list
    # (2): new_full_recfldgrn
    # (3): suffix
    ##############
    df_batch = pd.DataFrame([i['x'].to_dict() for i in batch_input])
    recfldgrn_sfx_list = [i for i in df_batch.columns]
    
    batch_rfg = {}

    for recfldgrn_sfx in recfldgrn_sfx_list:
        relational_list = df_batch[recfldgrn_sfx].to_list()
        B_recfldgrn_sfx = 'B-' + recfldgrn_sfx # B- means Batch. 
        suffix = '_' + B_recfldgrn_sfx.split('_')[-1]
        D = convert_relational_list_to_numpy(relational_list, B_recfldgrn_sfx, suffix)
        tensor_idx = D[B_recfldgrn_sfx]
        # print(B_recfldgrn_sfx, '<--- B_recfldgrn_suffix')
        # print(tensor_idx.shape, '<------- the shape of tensor_idx')
        batch_rfg[B_recfldgrn_sfx] = torch.Tensor(tensor_idx)
        # print('\n')
        
    batch_y = torch.LongTensor([i['y'] for i in batch_input])  # ignore this
    return batch_rfg, batch_y

In [40]:
batch_rfg, batch_y = my_collate_fn(batch_input)

In [41]:
# batch_rfg
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@PatEcInfo-InfoGrn_wgt torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx torch.Size([4, 87, 1, 15, 517])
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt torch.Size([4, 87, 1, 15, 517])
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx torch.Size([4, 87, 1, 15, 517])


In [42]:
batch_y

tensor([1, 0, 1, 0])

In [43]:
from torch.utils.data import DataLoader

my_dataloader = DataLoader(dataset, batch_size = 2, shuffle = True, collate_fn = my_collate_fn)

In [44]:
for idx, batch in enumerate(my_dataloader):
    
    print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    print(batch_y.shape)


------ 0
B-P@PatEcInfo-InfoGrn_wgt torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([2, 43])
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx torch.Size([2, 22, 1, 13, 221])
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt torch.Size([2, 22, 1, 13, 221])
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx torch.Size([2, 22, 1, 13, 221])
torch.Size([2])

------ 1
B-P@PatEcInfo-InfoGrn_wgt torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([2, 43])
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx torch.Size([2, 34, 1, 14, 199])
B-P-EC-PN-PNSect@AllText-TknzGrn_wgt torch.Size([2, 34, 1, 14, 199])
B-P-EC-PN-PNSect@AllText-TknzGrn_fldidx torch.Size([2, 34, 1, 14, 199])
torch.Size([2])

------ 2
B-P@PatEcInfo-InfoGrn_wgt torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([2, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([2, 43])
B-P-EC-PN-PNSect@AllText-TknzGrn_tknidx torch.Size([2, 26, 1, 14, 283])
B-P-EC-P